In [1]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2
import json
import os

In [2]:
files = os.listdir()
jsons = [file for file in files if not os.path.isdir(file) and '.json' in file]

In [3]:
db_config = {
    'host': '127.0.0.1',
    'database': 'postgres',
    'user': 'postgres',
    'password': '1234567890'
}

In [4]:
conn = psycopg2.connect(**db_config)

In [5]:
engine = create_engine(f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [ ]:
list_columns = []

In [ ]:
for json in jsons:
    temp_df = pd.read_json(json)
    json_list = temp_df['response']['docs']
    df = pd.DataFrame(json_list)
    
    columns = df.columns.tolist()
    def_columns = ', '.join([f'"{coluna}" text' for column in columns])
    
    temp_list = set(list_columns + def_columns) 
    list_columns = list(temp_list)
    
    del temp_df, json_list, df, temp_list, def_columns
    
    print(f"Colunas do {json} verificadas com sucesso!")

In [7]:
try:
    cursor = conn.cursor()
    query = f"CREATE TABLE IF NOT EXISTS oasisbr2 ({list_columns});"
    cursor.execute(query)
    conn.commit()

    print("Tabela criada com sucesso!")
    
except Exception as e:
    conn.rollback()
    print(f"Erro ao criar a tabela: {e}")
    
finally:
    cursor.close()
    conn.close()

Tabela criada com sucesso!


In [ ]:
for json in jsons:
    temp_df = pd.read_json(json)
    json_list = temp_df['response']['docs']
    df = pd.DataFrame(json_list)
    
    df.to_sql("oasisbr2", engine, if_exists='append', index=False)
    
    del temp_df, json_list, df
    
    print(f"{json} enviado ao banco com sucesso!")